- change early_stopping rounds to 50
- modify mistake in 

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, StratifiedKFold,TimeSeriesSplit,KFold,GroupKFold
from datetime import datetime
from kaggle.competitions import nflrush
import math
import tqdm
from scipy.special import expit
from scipy.spatial import Delaunay, delaunay_plot_2d, Voronoi, voronoi_plot_2d, ConvexHull
pd.set_option("display.max_rows",1000)
env = nflrush.make_env()

In [2]:
def crps_score(y_pred, data):
    y_pred = np.transpose(y_pred.reshape(199,-1))
    y_true = np.array(data.get_label()).astype("int")
    y_true_cdf = np.zeros([y_true.shape[0], 199])
    for i, y in enumerate(y_true):
        y_true_cdf[i, y] = 1
    y_true_cdf = np.clip(np.cumsum(y_true_cdf, axis=1), 0, 1)
    y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
    return "CRPS", ((y_true_cdf - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * y_true.shape[0]), False

def simple_crps(y_true, y_pred):
    y_true_cdf = np.zeros([y_true.shape[0], 199])
    for i, y in enumerate(y_true):
        y_true_cdf[i, y] = 1
    y_true_cdf = np.clip(np.cumsum(y_true_cdf, axis=1), 0, 1)
    y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
    return ((y_true_cdf - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * y_true.shape[0])

In [3]:
train = pd.read_csv('../input/nfl-big-data-bowl-2020/train.csv',low_memory=False)

# preprocess and feature engineering 

In [4]:
def transform_time_all(str1,quarter):
    if quarter<=4:
        return 15*60 - (int(str1[:2])*60 + int(str1[3:5])) + (quarter-1)*15*60
    if quarter ==5:
        return 10*60 - (int(str1[:2])*60 + int(str1[3:5])) + (quarter-1)*15*60
    
def transform_height(te):
    return (int(te.split('-')[0])*12 + int(te.split('-')[1]))*2.54/100

def voronoi_volumes(points, selected_index):
    if np.isnan(points).any(): # if missing value is included
        return -999
    
    else:
        v = Voronoi(points)
        vol = np.zeros(v.npoints)
      
        for i, reg_num in enumerate(v.point_region):
            if reg_num == v.point_region[selected_index]:
                indices = v.regions[reg_num]
                if -1 in indices: # some regions can be opened
                    vol = -999 ## insert missing value when the area is open
                else:
                    vol = ConvexHull(v.vertices[indices]).volume      
                break
        return vol

def radius_calc(dist_to_ball):
    return 4 + 6 * (dist_to_ball >= 15) + (dist_to_ball ** 3) / 560 * (dist_to_ball < 15)

def compute_influence_to_ball(row):
    point = np.array([row["RushX"], row["RushY"]])
    theta = math.radians(row['Orientation'])
    speed = row['S']
    player_coords = row[['X', 'Y']].values
    dist_to_ball = row["DisToRusher"]    

    S_ratio = (speed / 13) ** 2    # we set max_speed to 13 m/s ((dominance_df["S"]*0.9144).max() #8.64m/s)
    RADIUS = radius_calc(dist_to_ball)  # updated

    S_matrix = np.matrix([[RADIUS * (1 + S_ratio), 0], [0, RADIUS * (1 - S_ratio)]])
    R_matrix = np.matrix([[np.cos(theta), - np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    COV_matrix = np.dot(np.dot(np.dot(R_matrix, S_matrix), S_matrix), np.linalg.inv(R_matrix))
    
    norm_fact = (1 / 2 * np.pi) * (1 / np.sqrt(np.linalg.det(COV_matrix)))    
    mu_play = player_coords + speed * np.array([np.cos(theta), np.sin(theta)]) / 2
    
    intermed_scalar_player = np.dot(np.dot((player_coords - mu_play),
                                    np.linalg.inv(COV_matrix)),
                             np.transpose((player_coords - mu_play)))
    player_influence = norm_fact * np.exp(- 0.5 * intermed_scalar_player[0, 0])
    
    intermed_scalar_point = np.dot(np.dot((point - mu_play), 
                                    np.linalg.inv(COV_matrix)), 
                             np.transpose((point - mu_play)))
    point_influence = norm_fact * np.exp(- 0.5 * intermed_scalar_point[0, 0])
    
    return point_influence / player_influence

In [5]:
remove_features = ['GameId','PlayId','DisplayName','GameClock','TimeHandoff','TimeSnap', 'PlayDirection', 'TeamOnOffense', 
                    'PlayerBirthDate', 'is_run', 'NflIdRusher', 'date_game', 'RushX', 'RushY', 'PossessionTeam', 
                   'FieldPosition', 'Position', 'PlayerHeight', 'HomeTeamAbbr', 'VisitorTeamAbbr', 'Turf', 'Quarter'
                   , 'RushX_0.5s', 'RushY_0.5s', 'RushX_1s', 'RushY_1s', 'OffenseFormation', 'Team', 'WindSpeed',
                   'GameWeather', 'WindDirection', 'Location', 'Stadium', 'StadiumType', 'Temperature', 'DefensePersonnel'
                  , 'PlayerCollegeName', 'OffensePersonnel', 'Humidity']

In [6]:
def transform_data(df):
    df.loc[df.VisitorTeamAbbr == "ARI",'VisitorTeamAbbr'] = "ARZ"
    df.loc[df.HomeTeamAbbr == "ARI",'HomeTeamAbbr'] = "ARZ"

    df.loc[df.VisitorTeamAbbr == "BAL",'VisitorTeamAbbr'] = "BLT"
    df.loc[df.HomeTeamAbbr == "BAL",'HomeTeamAbbr'] = "BLT"

    df.loc[df.VisitorTeamAbbr == "CLE",'VisitorTeamAbbr'] = "CLV"
    df.loc[df.HomeTeamAbbr == "CLE",'HomeTeamAbbr'] = "CLV"

    df.loc[df.VisitorTeamAbbr == "HOU",'VisitorTeamAbbr'] = "HST"
    df.loc[df.HomeTeamAbbr == "HOU",'HomeTeamAbbr'] = "HST"

    df['is_run'] = df.NflId == df.NflIdRusher

    df['S'] = 10 * df['Dis']
    df.loc[df.Season == 2017, 'A'] = df[df.Season == 2018]['A'].mean()
    
    df['ToLeft'] = df.PlayDirection == "left"
    df['TeamOnOffense'] = "home"
    df.loc[df.PossessionTeam != df.HomeTeamAbbr, 'TeamOnOffense'] = "away"
    df['OnOffense'] = df.Team == df.TeamOnOffense 
    df['YardLine_std'] = 100 - df.YardLine.copy()
    df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
            'YardLine_std'
             ] = df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
              'YardLine']
    df['X_std'] = df.X.copy()
    df.loc[df.ToLeft, 'X_std'] = 120 - df.loc[df.ToLeft, 'X'] 
    df['Y_std'] = df.Y.copy()
    df.loc[df.ToLeft, 'Y_std'] = 53.3 - df.loc[df.ToLeft, 'Y'] 
    df['Orientation_std'] = df.Orientation.copy()
    df.loc[df.ToLeft, 'Orientation_std'] = np.mod(180 + df.loc[df.ToLeft, 'Orientation_std'], 360)
    df['Dir_std'] = df.Dir.copy()
    df.loc[df.ToLeft, 'Dir_std'] = np.mod(180 + df.loc[df.ToLeft, 'Dir_std'], 360)
    df.loc[df['Season'] == 2017, 'Orientation_std'] = np.mod(90 + df.loc[df['Season'] == 2017, 'Orientation_std'], 360) 
    df.drop(["X", "Y", "Orientation", "YardLine", "Dir", "ToLeft"], axis=1, inplace=True)
    df.rename(columns={'X_std': 'X', 'Y_std': 'Y', 'Orientation_std': 'Orientation', 'Dir_std': 'Dir', "YardLine_std": "YardLine"}, inplace=True)
    
    df['date_game'] = df.GameId.map(lambda x:pd.to_datetime(str(x)[:8]))
    df['age'] = (df.date_game.map(pd.to_datetime) - df.PlayerBirthDate.map(pd.to_datetime)).map(lambda x:x.days)/365

    df["Momentum"] = df["S"] * df["PlayerWeight"]
    
    df["Dir_cos"] = df["Dir"].apply(lambda x : np.cos((450-x) * np.pi/ 180))
    
    df["Momentum_cos"] = df["Momentum"] * df["Dir_cos"]
    
    df["Dir_sin"] = df["Dir"].apply(lambda x : np.sin((450-x) * np.pi/ 180))
    
    df["S_cos"] = df["S"] * df["Dir_cos"]
    df["S_sin"] = df["S"] * df["Dir_sin"]
    df["X_0.5s"] = df["X"] + df["S_cos"] * 0.5
    df["Y_0.5s"] = df["Y"] + df["S_sin"] * 0.5
    df["X_1s"] = df["X"] + df["S_cos"] * 1
    df["Y_1s"] = df["Y"] + df["S_sin"] * 1

    rusher_x = np.array(df.groupby(["PlayId", "is_run"])["X"].agg(np.mean)[1::2])
    df["RushX"] = np.repeat(rusher_x, 22) # repeat each elemnt 22 times
    rusher_y = np.array(df.groupby(["PlayId", "is_run"])["Y"].agg(np.mean)[1::2])
    df["RushY"] = np.repeat(rusher_y, 22) 
    df["DisToRusher"] = np.sqrt((df["X"] - df["RushX"]) ** 2 + (df["Y"] - df["RushY"]) ** 2)
    df["TackleTimeToRusher"] = df["DisToRusher"] / df["S"] # includes nan when the speed of rusher is 0
    
    rusher_x = np.array(df.groupby(["PlayId", "is_run"])["X_0.5s"].agg(np.mean)[1::2])
    df["RushX_0.5s"] = np.repeat(rusher_x, 22) 
    rusher_y = np.array(df.groupby(["PlayId", "is_run"])["Y_0.5s"].agg(np.mean)[1::2])
    df["RushY_0.5s"] = np.repeat(rusher_y, 22) 
    df["DisToRusher_0.5s"] = np.sqrt((df["X_0.5s"] - df["RushX_0.5s"]) ** 2 + (df["Y_0.5s"] - df["RushY_0.5s"]) ** 2)
    df["TackleTimeToRusher_0.5s"] = df["DisToRusher_0.5s"] / df["S"]

    rusher_x = np.array(df.groupby(["PlayId", "is_run"])["X_1s"].agg(np.mean)[1::2])
    df["RushX_1s"] = np.repeat(rusher_x, 22) 
    rusher_y = np.array(df.groupby(["PlayId", "is_run"])["Y_1s"].agg(np.mean)[1::2])
    df["RushY_1s"] = np.repeat(rusher_y, 22) 
    df["DisToRusher_1s"] = np.sqrt((df["X_1s"] - df["RushX_1s"]) ** 2 + (df["Y_1s"] - df["RushY_1s"]) ** 2)
    df["TackleTimeToRusher_1s"] = df["DisToRusher_1s"] / df["S"]
    
    #df["InfluToBall"] = df.apply(compute_influence_to_ball, axis=1)
    
    df["AttackAngle"] =  np.arctan((df["Y"] - df["RushY"]) / (df["X"] - df["RushX"])) * 180 / np.pi
    
    df_single = df[df.is_run==True].copy()
        
    df_single['time_end'] = df_single.apply(lambda x:transform_time_all(x.loc['GameClock'],x.loc['Quarter']),axis=1)
                                                                                                                                  
    df_single['DefendersInTheBox_vs_Distance'] = df_single['DefendersInTheBox'] / df_single['Distance']                                                           
  
    df_single["DisToQB"] = np.array(df[(df.Position=="QB") | (df.Position=="C")].groupby(["PlayId"]).agg(np.mean)["DisToRusher"])

    df_single['runner_height'] = df_single.PlayerHeight.map(transform_height)
    df_single.drop(remove_features,axis=1,inplace=True) 

    tmp = df.groupby(["PlayId", "OnOffense"]).agg(np.mean)[["X", "Y", "Momentum", "Momentum_cos"]]
    df_single["DefenseAveX"] = np.array(tmp[0::2]["X"])
    df_single["OffenseAveX"] = np.array(tmp[1::2]["X"])
    df_single["DefenseAveY"] = np.array(tmp[0::2]["Y"]) 
    df_single["OffenseAveY"] = np.array(tmp[1::2]["Y"]) 
    df_single["DefenseAveMomentum"] = np.array(tmp[0::2]["Momentum"])
    df_single["OffenseAveMomentum"] = np.array(tmp[1::2]["Momentum"])
    df_single["DefenseAveMomentum_cos"] = np.array(tmp[0::2]["Momentum_cos"])
    df_single["OffenseAveMomentum_cos"] = np.array(tmp[1::2]["Momentum_cos"])

    tmp = df.groupby(["PlayId", "OnOffense"]).agg(["std"])[["X", "Y", "Momentum", "Momentum_cos"]]
    df_single["DefenseStdX"] = np.array(tmp[0::2]["X"])
    df_single["OffenseStdX"] = np.array(tmp[1::2]["X"])
    df_single["DefenseStdY"] = np.array(tmp[0::2]["Y"])
    df_single["OffenseStdY"] = np.array(tmp[1::2]["Y"])
    df_single["DefenseStdMomentum"] = np.array(tmp[0::2]["Momentum"])
    df_single["OffenseStdMomentum"] = np.array(tmp[1::2]["Momentum"])
    df_single["DefenseStdMomentum_cos"] = np.array(tmp[0::2]["Momentum_cos"])
    df_single["OffenseStdMomentum_cos"] = np.array(tmp[1::2]["Momentum_cos"])
    
    tmp = df.groupby(["PlayId", "OnOffense"]).agg(["std"])[["X_0.5s", "Y_0.5s", "X_1s", "Y_1s"]]
    df_single["DefenseStdX_0.5s"] = np.array(tmp[0::2]["X_0.5s"])
    df_single["OffenseStdX_0.5s"] = np.array(tmp[1::2]["X_0.5s"])
    df_single["DefenseStdY_0.5s"] = np.array(tmp[0::2]["Y_0.5s"]) 
    df_single["OffenseStdY_0.5s"] = np.array(tmp[1::2]["Y_0.5s"]) 
    df_single["DefenseStdX_1s"] = np.array(tmp[0::2]["X_1s"])
    df_single["OffenseStdX_1s"] = np.array(tmp[1::2]["X_1s"])
    df_single["DefenseStdY_1s"] = np.array(tmp[0::2]["Y_1s"]) 
    df_single["OffenseStdY_1s"] = np.array(tmp[1::2]["Y_1s"])
    
    df_single["RunnerToDefenseCentoid"] = np.sqrt((df_single["X"] - df_single["DefenseAveX"]) ** 2 + (df_single["Y"] - df_single["DefenseAveY"]) ** 2)
    df_single["RunnerToOffenseCentoid"] = np.sqrt((df_single["X"] - df_single["OffenseAveX"]) ** 2 + (df_single["Y"] - df_single["OffenseAveY"]) ** 2)
    
    df_single["RunnerToScrimmage"] = df_single["X"] - (df_single["YardLine"] + 10)

    df_single["MinTackleTime"] = np.array(df.groupby(["PlayId", "OnOffense"])["TackleTimeToRusher"].min()[0::2])
    df_single["MinTackleTime_0.5s"] = np.array(df.groupby(["PlayId", "OnOffense"])["TackleTimeToRusher_0.5s"].min()[0::2])
    df_single["MinTackleTime_1s"] = np.array(df.groupby(["PlayId", "OnOffense"])["TackleTimeToRusher_1s"].min()[0::2])
    df_single["1stDefender_Momentum_cos"] = np.array(df.loc[df.groupby(["PlayId", "OnOffense"])["DisToRusher"].idxmin()[0::2]]["Momentum_cos"])
    first_offenser_index =df.groupby(["PlayId", "OnOffense"])["DisToRusher"].nsmallest(2)[3::4].reset_index()["level_2"]
    df_single["1stOffenser_Momentum_cos"] = np.array(df.loc[first_offenser_index]["Momentum_cos"])
    df_single["1stDefender_AttackAngle"] = np.array(df.loc[df.groupby(["PlayId", "OnOffense"])["DisToRusher"].idxmin()[0::2]]["AttackAngle"])    
    
    #def_influ = np.array(df.groupby(["PlayId", "OnOffense"]).agg("sum")["InfluToBall"][0::2])
    #off_influ = np.array(df.groupby(["PlayId", "OnOffense"]).agg("sum")["InfluToBall"][1::2])
    #df_single["PitchControl"] = expit(off_influ - def_influ)
    
    df_single.fillna(-999,inplace=True) 
    remove_features2 = ["OnOffense", "DisToRusher", "TackleTimeToRusher", "OffenseAveX", "OffenseAveY", "AttackAngle", "Dis", 
                    "DisToRusher_0.5s", "TackleTimeToRusher_0.5s", "DisToRusher_1s", "TackleTimeToRusher_1s"]
    df_single.drop(remove_features2, axis=1, inplace=True)

    return df_single

In [7]:
%%time
train_single = transform_data(train)
y_train = train_single.Yards + 99 # to categorize
X_train = train_single.drop(['Yards'],axis=1)
for f in X_train.columns:
    if X_train[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f])+[-999])
        X_train[f] = lbl.transform(list(X_train[f]))

CPU times: user 7min 47s, sys: 4.53 s, total: 7min 52s
Wall time: 7min 47s


# modelling

In [8]:
# modified CV
df_20172018 = pd.concat([X_train, y_train], axis=1)
df_2017 = df_20172018[df_20172018.Season==2017].copy()
df_2018 = df_20172018[df_20172018.Season==2018].copy()
X_2017 = df_2017.drop(['Yards'],axis=1)
y_2017 = df_2017.Yards
X_2018 = df_2018.drop(['Yards'],axis=1)
y_2018 = df_2018.Yards
groups = np.array(X_2018.Week)
n_folds=3
gkf=GroupKFold(n_splits = n_folds)

average_crps = 0
models = []
crps_scores = []
lgbm_params = {
    "objective" : "multiclass",
    "metric" : "None", # if set None, only custom metric is considered.
    "tree_learner": "serial",
    "max_depth" : 5,
    "boosting": 'gbdt',
    "num_class": 199,
    "num_leaves" : 13,
    "learning_rate" : 0.01,
}
evals_result = {}
feature_importance_df = pd.DataFrame(list(X_train.columns), columns=["Feature"])
for i , (train_index, test_index) in enumerate(gkf.split(X_2018, y_2018, groups)):
    X_train_2018 = X_2018.iloc[train_index,:]
    y_train_2018 = y_2018.iloc[train_index]
    X_test2 = X_2018.iloc[test_index,:]
    y_test2 = y_2018.iloc[test_index]
    X_train2 = pd.concat([X_2017, X_train_2018])
    y_train2 = pd.concat([y_2017, y_train_2018])
    lgb_train = lgb.Dataset(X_train2, y_train2)
    lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
    
    clf = lgb.train(
        lgbm_params, lgb_train,
        valid_sets=[lgb_train, lgb_eval],
        num_boost_round=100000,
        early_stopping_rounds=50,
        evals_result=evals_result,
        feval=crps_score,
        verbose_eval = 100,
    )
    
    models.append(clf)
    temp_predict = clf.predict(X_test2, num_iteration = clf.best_iteration)
    score = simple_crps(y_test2, temp_predict)
    print(score)
    crps_scores.append(score)
    average_crps += score / n_folds
    feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()
    
feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]
print('crps list:', crps_scores)
print('average crps:',average_crps)

Training until validation scores don't improve for 50 rounds
[100]	training's CRPS: 0.0118616	valid_1's CRPS: 0.0140845
[200]	training's CRPS: 0.0106106	valid_1's CRPS: 0.0138399
[300]	training's CRPS: 0.00985414	valid_1's CRPS: 0.0137217
[400]	training's CRPS: 0.00932935	valid_1's CRPS: 0.0136608
[500]	training's CRPS: 0.00891434	valid_1's CRPS: 0.0136324
[600]	training's CRPS: 0.00857496	valid_1's CRPS: 0.013619
[700]	training's CRPS: 0.00827315	valid_1's CRPS: 0.0136161
Early stopping, best iteration is:
[685]	training's CRPS: 0.00831601	valid_1's CRPS: 0.0136154
0.01361543083786209
Training until validation scores don't improve for 50 rounds
[100]	training's CRPS: 0.0119143	valid_1's CRPS: 0.0140649
[200]	training's CRPS: 0.0106837	valid_1's CRPS: 0.0138075
[300]	training's CRPS: 0.00994673	valid_1's CRPS: 0.0136835
[400]	training's CRPS: 0.00944133	valid_1's CRPS: 0.0136162
[500]	training's CRPS: 0.00905181	valid_1's CRPS: 0.01358
[600]	training's CRPS: 0.00871872	valid_1's CRPS: 

In [9]:
X_train.columns

Index(['S', 'A', 'NflId', 'JerseyNumber', 'Season', 'Down', 'Distance',
       'HomeScoreBeforePlay', 'VisitorScoreBeforePlay', 'DefendersInTheBox',
       'PlayerWeight', 'Week', 'YardLine', 'X', 'Y', 'Orientation', 'Dir',
       'age', 'Momentum', 'Dir_cos', 'Momentum_cos', 'Dir_sin', 'S_cos',
       'S_sin', 'X_0.5s', 'Y_0.5s', 'X_1s', 'Y_1s', 'time_end',
       'DefendersInTheBox_vs_Distance', 'DisToQB', 'runner_height',
       'DefenseAveX', 'DefenseAveY', 'DefenseAveMomentum',
       'OffenseAveMomentum', 'DefenseAveMomentum_cos',
       'OffenseAveMomentum_cos', 'DefenseStdX', 'OffenseStdX', 'DefenseStdY',
       'OffenseStdY', 'DefenseStdMomentum', 'OffenseStdMomentum',
       'DefenseStdMomentum_cos', 'OffenseStdMomentum_cos', 'DefenseStdX_0.5s',
       'OffenseStdX_0.5s', 'DefenseStdY_0.5s', 'OffenseStdY_0.5s',
       'DefenseStdX_1s', 'OffenseStdX_1s', 'DefenseStdY_1s', 'OffenseStdY_1s',
       'RunnerToDefenseCentoid', 'RunnerToOffenseCentoid', 'RunnerToScrimmage',
       '

In [10]:
feature_importance_df.sort_values("Average").head(100).reset_index(drop=True)

,Feature,Fold_1,Fold_2,Fold_3,Average,Std,Cv
0,Season,1061,997,573,877.000000,216.542529,0.246913
1,Down,1789,2033,2483,2101.666667,287.454731,0.136775
2,DefendersInTheBox,4229,4673,4906,4602.666667,280.822997,0.061013
3,X_0.5s,5304,6097,4662,5354.333333,586.916424,0.109615
4,DefenseStdY_0.5s,4612,5563,6375,5516.666667,720.487027,0.130602
5,runner_height,5571,6804,6884,6419.666667,600.986042,0.093616
6,S_sin,6868,6592,7867,7109.000000,547.702474,0.077044
7,Week,6004,7898,7571,7157.666667,826.616531,0.115487
8,X_1s,7324,7711,6941,7325.333333,314.352598,0.042913
9,Distance,8347,5294,9212,7617.666667,1680.600356,0.220619


# prediction

In [11]:
for (test_df, sample_prediction_df) in env.iter_test():
    X_test = transform_data(test_df)
    for f in X_test.columns:
        if X_test[f].dtype=='object':
            X_test[f] = X_test[f].map(lambda x:x if x in set(X_train[f]) else -999)
    for f in X_test.columns:
        if X_test[f].dtype=='object': 
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(X_train[f])+[-999])
            X_test[f] = lbl.transform(list(X_test[f])) 
    pred_value = 0
    for model in models:
        pred_value += model.predict(X_test, num_iteration = model.best_iteration)[0]/ n_folds
    pred_data = np.clip(np.cumsum(pred_value), 0, 1)
    pred_data = np.array(pred_data).reshape(1,199)
    pred_target = pd.DataFrame(index = sample_prediction_df.index, \
                               columns = sample_prediction_df.columns, \
                               data = pred_data)
    env.predict(pred_target)
env.write_submission_file()

Your submission file has been saved!  Once you `Commit` your Notebook and it finishes running, you can submit the file to the competition from the Notebook Viewer `Output` tab.
